In [ ]:
!pip install soynlp
!pip install konlpy
!pip install sentence_transformers

In [ ]:
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

In [27]:
import pandas as pd
import numpy as np
import itertools
from soynlp.noun import LRNounExtractor_v2
from soynlp.tokenizer import LTokenizer
from konlpy.tag import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import re

## Soynlp

In [67]:
bank_fraud = []
for i in range(0, 135) :
  with open(f'/content/drive/MyDrive/Colab Notebooks/LIFLOW/data/voice_phishing/대출사기/{i}.txt', 'r', encoding='UTF8') as f :
    tmp = f.readlines()
    bank_fraud += tmp

impersonation = []
for i in range(0, 57) :
  with open(f'/content/drive/MyDrive/Colab Notebooks/LIFLOW/data/voice_phishing/사칭/{i}.txt', 'r', encoding='UTF8') as f :
    tmp = f.readlines()
    impersonation += tmp

print(len(bank_fraud))
print(len(impersonation))

7566
7939


In [68]:
tmp_bank_fraud = []
tmp_impersonation = []

for i in range(len(bank_fraud)) :
  if bank_fraud[i][:3] == '사기범' :
    tmp_bank_fraud.append(bank_fraud[i][6:-1])
for i in range(len(impersonation)) :
  if impersonation[i][:3] == '사기범' :
      tmp_impersonation.append(impersonation[i][6:-1])
bank_fraud = tmp_bank_fraud[:]
impersonation = tmp_impersonation[:]
print(len(bank_fraud))
print(len(impersonation))

3725
3796


In [69]:
bank_fraud = [re.sub('[^ㄱ-힣 ]', '', s) for s in bank_fraud]
impersonation = [re.sub('[^ㄱ-힣 ]', '', s) for s in impersonation]

bank_fraud = [re.sub('[ㄱ-ㅎ]', '', s) for s in bank_fraud]
impersonation = [re.sub('[ㄱ-ㅎ]', '', s) for s in impersonation]

In [54]:
"""
from konlpy.tag import *
okt = Okt()
bank_fraud_tokens = [okt.nouns(s) for s in bank_fraud]
impersonation_tokens = [okt.nouns(s) for s in impersonation]
print(bank_fraud_tokens[:5])
print(impersonation_tokens[:5])
"""

'\nfrom konlpy.tag import *\nokt = Okt()\nbank_fraud_tokens = [okt.nouns(s) for s in bank_fraud]\nimpersonation_tokens = [okt.nouns(s) for s in impersonation]\nprint(bank_fraud_tokens[:5])\nprint(impersonation_tokens[:5])\n'

In [71]:
doc1 = ' '.join(bank_fraud)
doc2 = ' '.join(impersonation)

In [72]:
noun_extractor1 = LRNounExtractor_v2()
nouns1 = noun_extractor1.train_extract(bank_fraud)

noun_extractor2 = LRNounExtractor_v2()
nouns2 = noun_extractor2.train_extract(impersonation)

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=3929, neg=2321, common=107
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 10916 from 3725 sents. mem=0.866 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=46264, mem=0.866 Gb
[Noun Extractor] batch prediction was completed for 2883 words
[Noun Extractor] checked compounds. discovered 240 compounds
[Noun Extractor] postprocessing detaching_features : 1338 -> 1276
[Noun Extractor] postprocessing ignore_features : 1276 -> 1234
[Noun Extractor] postprocessing ignore_NJ : 1234 -> 1231
[Noun Extractor] 1231 nouns (240 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=0.866 Gb                    
[Noun Extractor] 55.59 % eojeols are covered
[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=3929, neg=2321, common=107
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 11981 from 3

In [73]:
noun_score1 = dict([])
for w, f in nouns1.items() :
  noun_score1[w] = nouns1[w].score

noun_score2 = dict([])
for w, f in nouns2.items() :
  noun_score2[w] = nouns2[w].score

In [74]:
tokenizer1 = LTokenizer(scores = noun_score1)
tokenizer2 = LTokenizer(scores = noun_score2)

In [75]:
tokenized_nouns1 = tokenizer1.tokenize(doc1)
tokenized_nouns1 = ' '.join(tokenized_nouns1)
print(tokenized_nouns1)

tokenized_nouns2 = tokenizer2.tokenize(doc2)
tokenized_nouns2 = ' '.join(tokenized_nouns2)
print(tokenized_nouns2)

네 삐 삐 님 어저께 통화 했던 최영환입니다 어떻게 가진행 결과 안내 차 오늘 아침 에 전화 드린다고 제가 얘기 를 했었는데 어떻게 통화 는 괜찮으세요 일단 최대한도 는 고객 님 신청 하신 최대 한도 는 천만원 까지 가능 하시고요 금리 는 연 금리 에 상환 기간 개월 입니다 년이고요 매 월 상환 하시는 금액 은 개월 기준 원금 이자 포함 되신 금액 으로 원 확인되시는데 괜찮으세요 이자 입니다 금리 가 괜찮으세요 아니 괜찮으시냐고요 그럼 간단 하게 상품 설명 부터 드릴게요 현재 진행 하시는 대출 상품 은 신용 대출 은 아니고 소득 담보 대출 이라 그래서 연체 를 하실 경우 나 고의적 인 상환 회피 가 있으실 경우 향후 고객 님 발생 하시는 미래에셋 쪽 부분 에 대한 채권 추심 에 대한 권한 을 부여 받고 진행 이 되시면 대출 상품 입니다 괜찮으신가요 이 효력 을 보유 하는 과정 으로는 직접 고객 님께서 법무법인이나 변호사 공중 사무실 을 지정 해 주셔서 저희 은행 직원 파견 나가시는 분을 만나 뵙고 자대면을 통해서 대출 약정서와 공증서를 작성 하게 되구요 뭐 이게 좀 어려우실 경우 에는 저희 은행 쪽으로 위임 처리 하시는 방법 도 있으시니까 어떻게 뭐 어떤 방법 으로 진행 을 도와 드릴까요 만약 에 지역 이 지금 어디죠 지방 같은 경우 는 시간 이 좀 걸려요 한 일주일 정도 위임 처리 하시면 당일 로도 가능 하시고 위임 하실 경우 에는 방문 이나 내방 없이 드리는 전화 받으시고 녹취 를 통해서 대출 계약서 랑 공증 서류 를 작성 하고요 우편 으로 등기 발송 처리 해드리고요 그렇죠 당일 로도 가능 하시고 다만 위임 처리 하실 경우 에는 이제 뭐 수수료 는 저희 가 대신 내드리지만 위임 처리 비용 이 따로 발생 이 되세요 만원 이요 고정 금리 죠 중도 상환 수수료 는 최소 개월 동안 만 연체 없으시면 중도 상환 수수료 안 받습니다 없습니다 그러면은 이제 고객 님께서 직접 법무 법인 이랑 변호사 공증 사무실 지정 하고 저희 은행 직원 이 파견 나가는 걸로 해드릴까요 

In [81]:
#n_gram_range = (1,)
#n_gram_range = (2,3)
#count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
count1 = CountVectorizer().fit([tokenized_nouns1])
candidates1 = count1.get_feature_names_out()

print('trigram 개수 :',len(candidates1))
print('trigram 다섯개만 출력 :',candidates1[:5])

count2 = CountVectorizer().fit([tokenized_nouns2])
candidates2 = count2.get_feature_names_out()

print('trigram 개수 :',len(candidates2))
print('trigram 다섯개만 출력 :',candidates2[:5])

trigram 개수 : 7031
trigram 다섯개만 출력 : ['가가지고' '가거나' '가격' '가결' '가결나신']
trigram 개수 : 7335
trigram 다섯개만 출력 : ['가게' '가고' '가기' '가까' '가까우셔서']


In [82]:
model1 = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
doc_embedding1 = model1.encode([doc1])
candidate_embeddings1 = model1.encode(candidates1)

model2 = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
doc_embedding2 = model2.encode([doc2])
candidate_embeddings2 = model2.encode(candidates2)

Downloading:   0%|          | 0.00/574 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/731 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/527 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [84]:
top_n = 20
distances1 = cosine_similarity(doc_embedding1, candidate_embeddings1)
keywords1 = [candidates1[index] for index in distances1.argsort()[0][-top_n:]]
print(keywords1)

top_n = 20
distances2 = cosine_similarity(doc_embedding2, candidate_embeddings2)
keywords2 = [candidates2[index] for index in distances2.argsort()[0][-top_n:]]
print(keywords2)

['보내드릴까요', '욕하셨잖아요', '빨간날이였잖아요', '드려놓을게요', '해주셨잖아요', '채무자시잖아요', '노하우라든지', '말해주셨잖아요', '시켜드렸거든요', '힘드셨잖아요', '식목일이잖아요', '여쭤봐야죠', '해드렸잖아요', '빌려주셨겠죠', '밝혀드렸잖아요', '떨궈주시면은요', '갚으셨잖아요', '정해드려보겠습니다', '시켜드리잖아요', '다시한번확인부탁드릴게요']
['해드릴게요', '알려드릴게요라고', '심사하고', '보내드릴까요', '바쁜분이니까', '힘들었죠', '취해드릴까요', '받으셨겠지만', '시켜드릴게요', '받았다라든가', '일천오백육십팔만이에요', '시켜야겠죠', '검찰청', '해주셨잖아요', '시켜놓겠습니다', '해야되잖아요', '오만원권하고', '따라가는지', '검찰조사', '책임져드릴까요']


## Okt

In [85]:
bank_fraud = []
for i in range(0, 135) :
  with open(f'/content/drive/MyDrive/Colab Notebooks/LIFLOW/data/voice_phishing/대출사기/{i}.txt', 'r', encoding='UTF8') as f :
    tmp = f.readlines()
    bank_fraud += tmp

impersonation = []
for i in range(0, 57) :
  with open(f'/content/drive/MyDrive/Colab Notebooks/LIFLOW/data/voice_phishing/사칭/{i}.txt', 'r', encoding='UTF8') as f :
    tmp = f.readlines()
    impersonation += tmp

print(len(bank_fraud))
print(len(impersonation))

7566
7939


In [86]:
tmp_bank_fraud = []
tmp_impersonation = []

for i in range(len(bank_fraud)) :
  if bank_fraud[i][:3] == '사기범' :
    tmp_bank_fraud.append(bank_fraud[i][6:-1])
for i in range(len(impersonation)) :
  if impersonation[i][:3] == '사기범' :
      tmp_impersonation.append(impersonation[i][6:-1])
bank_fraud = tmp_bank_fraud[:]
impersonation = tmp_impersonation[:]
print(len(bank_fraud))
print(len(impersonation))

3725
3796


In [87]:
bank_fraud = [re.sub('[^ㄱ-힣 ]', '', s) for s in bank_fraud]
impersonation = [re.sub('[^ㄱ-힣 ]', '', s) for s in impersonation]

bank_fraud = [re.sub('[ㄱ-ㅎ]', '', s) for s in bank_fraud]
impersonation = [re.sub('[ㄱ-ㅎ]', '', s) for s in impersonation]

In [88]:
doc1 = ' '.join(bank_fraud)
doc2 = ' '.join(impersonation)

In [89]:
from konlpy.tag import *
okt = Okt()
tokenized_nouns1 = [okt.nouns(s) for s in bank_fraud]
tokenized_nouns2 = [okt.nouns(s) for s in impersonation]
print(tokenized_nouns1[:5])
print(tokenized_nouns2[:5])

[['네', '삐', '삐'], ['어저께', '통화', '최영환'], ['진행', '결과', '오늘', '아침', '전화', '제', '얘기', '통화'], ['일단', '최대', '한도', '고객', '신청', '최대', '한도', '천만원'], ['금리', '연', '금리', '상환', '기간', '개월', '년', '고요']]
[['서울', '중앙', '검', '첨단', '범죄', '수사', '일부', '이희', '사관', '사건', '조사', '법원', '증거', '제출', '용', '데', '녹취', '조사', '과정', '허위', '진술', '본인', '내용', '은닉', '본인', '이', '사건', '무관', '위증', '공무', '집행', '방해', '죄', '가중', '처벌', '수가', '민', '형사', '책임', '사실', '입', '각하', '좀'], ['네', '씨', '지금', '제', '본인', '확인', '한번', '생년', '월일', '앞', '여섯', '자리', '말씀', '부탁'], ['네', '성함'], ['네', '본인', '김성오', '일당', '알'], ['전혀', '진술']]


In [95]:
tmp1 = []
for i in range(len(tokenized_nouns1)) :
  tmp1 += tokenized_nouns1[i]

tmp2 = []
for i in range(len(tokenized_nouns2)) :
  tmp2 += tokenized_nouns2[i]

tokenized_nouns1 = ' '.join(tmp1)
tokenized_nouns2 = ' '.join(tmp2)

In [96]:
#n_gram_range = (1,)
#n_gram_range = (2,3)
#count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
count1 = CountVectorizer().fit([tokenized_nouns1])
candidates1 = count1.get_feature_names_out()

print('trigram 개수 :',len(candidates1))
print('trigram 다섯개만 출력 :',candidates1[:5])

count2 = CountVectorizer().fit([tokenized_nouns2])
candidates2 = count2.get_feature_names_out()

print('trigram 개수 :',len(candidates2))
print('trigram 다섯개만 출력 :',candidates2[:5])

trigram 개수 : 2167
trigram 다섯개만 출력 : ['가가' '가격' '가결' '가계' '가끔']
trigram 개수 : 2452
trigram 다섯개만 출력 : ['가게' '가기' '가까이' '가끔' '가능']


In [97]:
model1 = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
doc_embedding1 = model1.encode([doc1])
candidate_embeddings1 = model1.encode(candidates1)

model2 = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
doc_embedding2 = model2.encode([doc2])
candidate_embeddings2 = model2.encode(candidates2)

In [99]:
top_n = 10
distances1 = cosine_similarity(doc_embedding1, candidate_embeddings1)
keywords1 = [candidates1[index] for index in distances1.argsort()[0][-top_n:]]
print(keywords1)

top_n = 10
distances2 = cosine_similarity(doc_embedding2, candidate_embeddings2)
keywords2 = [candidates2[index] for index in distances2.argsort()[0][-top_n:]]
print(keywords2)

['먼저', '현금화', '고객', '활용', '천이백만원', '백사십만원', '천백만원', '수수료', '경명대로', '할랬는데']
['증언', '소환조사', '소송', '법죄', '재판', '검찰관', '정의', '검찰', '검찰청', '검찰조사']
